In [66]:
import pandas as pd
import numpy as np
from numpy import nan 
import random

In [67]:
order_df = pd.read_csv('order.table.xls',header=0,sep='\t')
dna_df = pd.read_csv('DNA.table.xls',header=0,sep='\t')
delivery_df = pd.read_csv('delivery.table.xls',header=0,sep='\t')
 

In [68]:
# order_df
# dna_df
# delivery_df

In [69]:
order_df['receive_date'] = delivery_df['receive_date']

In [70]:
# order_df['receive_date'] 

In [71]:
order_df['receive_date'] = order_df['receive_date'].str.strip(' 00:00:00').str.replace('-','.')

In [72]:
# 部分样品编码含有反斜杠，这些反斜杠在数据保存的路径被‘-’代替了 
order_df['sample_code_format'] =  order_df['sample_code'].str.replace('/','-').str.replace(' ','-').str.replace(u'加急','jiaji')

In [73]:
order_df.iloc[1958,:]

sample_code                  ZBAD2021-加急1
detection_items                飞鸟残骸鉴定（加急）
department                       北京大兴国际机场
order_date            2021-04-28 00:00:00
order_num                             NaN
report_date                           NaN
conclusion                            NaN
receive_date                    2021.04.3
sample_code_format        ZBAD2021-jiaji1
Name: 1958, dtype: object

In [74]:
# 找到那些不含有日期的字符，使他为nan：
order_df[order_df['receive_date'] == '.'] = nan
# 填充空缺的日期：
order_df['receive_date'].fillna(method='ffill',axis=0,inplace=True)



In [75]:
# order_df[order_df['receive_date'].str.contains('\b\.\b')]

# 由于日期格式不统一，所以 将字符串都转为日期类型：
order_df['receive_date'] = order_df['receive_date'].apply(pd.to_datetime,infer_datetime_format=True)


In [76]:
# order_df['receive_date'].apply(pd.to_datetime,infer_datetime_format=True)
# 再将日期转回字符串：
order_df['receive_date'] = order_df['receive_date'].apply(lambda x : x.strftime('%Y%m%d'))

In [77]:
length_df = len(order_df['receive_date'])
order_df['random_num'] = [str(random.randint(10000, 99999)) for i in range(length_df) ]

In [78]:
# order_df.random_num
order_df['order_num'] = order_df['receive_date']+ order_df['random_num'] 

# order_df['order_num'] 

In [79]:
# 由于有部分测序路径的文件名是以航班号而非样本名来命名，所以有必要吧航班号放进来：
order_df['event_flight']  = delivery_df['event_flight']

In [80]:
# 测试填空值：
# df = pd.DataFrame(np.random.randn(6,3))
# df.iloc[1:4,1] = nan
# df.iloc[2:5,2] = nan
# df.fillna(method='ffill',axis=0)

In [81]:
order_df['id'] = order_df.index +1 
# del order_df['random_num']
# del order_df['receive_date']
order_df.to_csv('order_df.tsv',encoding='utf_8_sig',index=False)
print('---1')

---1


In [82]:
#有些样品是没有样品名的，先挑出来，以后在处理这些样品。
order_df_null_samplecode = order_df[order_df['sample_code'].isnull()]
order_df_null_samplecode.to_csv('order_df_null_samplecode.tsv',encoding='utf_8_sig',index=False)

In [83]:

# order_df.loc[[8,9,10],'sample_code']



In [84]:
dna_df.to_csv('dna_df.tsv',encoding='utf_8_sig',index=False)

In [85]:
delivery_df.to_csv('delivery_df.tsv',encoding='utf_8_sig',index=False)

In [86]:
path_df = pd.read_csv('airport.csv',header=0,sep=',')

In [87]:
path_df.head

<bound method NDFrame.head of       Unnamed: 0      date  code gene_name  \
0              0  20200427  ZSHC      cytb   
1              1  20200427  ZSHC      cytb   
2              2  20200427  ZSHC      cytb   
3              3  20200427  JYCS      cytb   
4              4  20200427  ZSHC      cytb   
...          ...       ...   ...       ...   
2581        2581  20200825    KN      cytb   
2582        2582  20200825    MF      cytb   
2583        2583  20200825    CZ      cytb   
2584        2584  20200413  JYCS      cytb   
2585        2585  20200413  JYCS     COI-1   

                                              file_name  \
0     ZSHC-2020-07_L14995-H16065_C2291.L14995.PW0050...   
1     ZSHC-2020-04_L14995-H16065_C2291.L14995.PW0050...   
2     ZSHC-2020-02_L14995-H16065_C2291.L14995.PW0050...   
3     JYCS2020-03_L14995-H16065_C2291.L14995.PW00509...   
4     ZSHC-2020-03_L14995-H16065_C2291.L14995.PW0050...   
...                                                 ...   
2581

In [88]:
result_df = pd.merge(order_df,path_df,left_on=['sample_code_format'],right_on=['sample_code'],how='outer')
result_df_event_flight = pd.merge(order_df,path_df,left_on=['event_flight'],right_on=['sample_code'],how='inner') # 

In [89]:
result_df.to_csv('result_df.tsv',encoding='utf_8_sig',index=False)
result_df_event_flight.to_csv('result_df_event_flight.tsv',encoding='utf_8_sig',index=False)

In [90]:
result_df

,sample_code_x,detection_items,department,order_date,order_num,report_date,conclusion,receive_date,sample_code_format,random_num,event_flight,id,Unnamed: 0,date,code,gene_name,file_name,path,sample_code_y
0,ZSHC/2017/81,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,2018041664591,NaN,NaN,20180416,ZSHC-2017-81,64591,9C8555,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZSHC/2017/80,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,2018041664812,NaN,NaN,20180416,ZSHC-2017-80,64812,JD0472,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZSHC/2017/82,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,2018041645361,NaN,NaN,20180416,ZSHC-2017-82,45361,MF8026,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ZSHC/2017/83,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,2018041640074,NaN,NaN,20180416,ZSHC-2017-83,40074,JD5351,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ZSHC/2018/01,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,2018041679772,NaN,NaN,20180416,ZSHC-2018-01,79772,CA1704,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2535.0,20200825.0,NS,COI-1,NS8024-0205_COI-1_C2777.COI-1-bird-F.PW0090105...,bird_strike/20200825&0826萧山机场4个&北京大兴机场44个样品结果/...,NS8024-0205
3391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2570.0,20200825.0,NS,cytb,NS8024-0205_L14995-H16065_C2777.L14995.PW00902...,bird_strike/20200825&0826萧山机场4个&北京大兴机场44个样品结果/...,NS8024-0205
3392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2556.0,20200825.0,KN,cytb,KN5812-2020-8-18_L14995-H16065_C2777.L14995.PW...,bird_strike/20200825&0826萧山机场4个&北京大兴机场44个样品结果/...,KN5812-2020-8-18
3393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2571.0,20200825.0,KN,cytb,KN5622-2020-8-16_L14995-H16065_C2777.L14995.PW...,bird_strike/20200825&0826萧山机场4个&北京大兴机场44个样品结果/...,KN5622-2020-8-16


In [91]:
s = 'ZBAD2021-加急1'
s.replace('加急','jiaji')

'ZBAD2021-jiaji1'

In [92]:
# s

In [93]:
# order_df.iloc[[1849,1850],:]

In [94]:
print('--')

--


In [95]:
result_df.path

0                                                     NaN
1                                                     NaN
2                                                     NaN
3                                                     NaN
4                                                     NaN
                              ...                        
3390    bird_strike/20200825&0826萧山机场4个&北京大兴机场44个样品结果/...
3391    bird_strike/20200825&0826萧山机场4个&北京大兴机场44个样品结果/...
3392    bird_strike/20200825&0826萧山机场4个&北京大兴机场44个样品结果/...
3393    bird_strike/20200825&0826萧山机场4个&北京大兴机场44个样品结果/...
3394    bird_strike/20200825&0826萧山机场4个&北京大兴机场44个样品结果/...
Name: path, Length: 3395, dtype: object

In [96]:
from app import create_app
app = create_app()
app_context =  app.app_context()
app_context.push()



from app.models1 import *
from app.models import *
import random
from datetime import datetime

In [97]:
result_df['id'] = result_df['id'].apply(lambda x : int(x) if not pd.isnull(x) else 0)
# 导入数据库

In [139]:

from app import create_app
app = create_app()
app_context =  app.app_context()
app_context.push()

from app.models1 import *
from app.models import *
import random
from datetime import datetime


In [98]:
# result_df['create_time'] = pd.to_datetime(result_df['create_time'],infer_datetime_format=True )
# result_df['sample_date'] = pd.to_datetime(result_df['sample_date'],infer_datetime_format=True )
# result_df['receive_date'] = pd.to_datetime(result_df['receive_date'],infer_datetime_format=True )
# result_df['report_date'] = pd.to_datetime(result_df['report_date'],infer_datetime_format=True )
# print(type(result_df['order_date']))
# result_df['id'] = result_df['id'].apply(lambda x :  int(x)  if x else None  )
# result_df['id'] = result_df['id'].astype('int64')
# print(result_df.id )

print(result_df['id'])

0       1
1       2
2       3
3       4
4       5
       ..
3390    0
3391    0
3392    0
3393    0
3394    0
Name: id, Length: 3395, dtype: int64




old_id = None
i = 0
# lst = []
for ix,val in result_df.iterrows():
    data = val.to_dict()
    id =  data.get('id')
    if not id :
        continue   
    else:
#         print(val.get('id'))        
        
        o = OrderSeq.query.filter_by(id=id).first()
        if not o:
            o = OrderSeq()
            o.from_dict(data)
            db.session.add(o)
        
    if val.get('path') and not pd.isnull(val.get('path')):
        seqData = SeqData()
        seqData.from_dict(data)
 
        with open (val.get('path')) as fh:
            raw_data = fh.read()
            seqData.raw_data = raw_data 
            
        o.seqData.append(seqData)
        db.session.add(seqData)
    
#     old_id = data.get('id')
    
    i+= 1
    if i%100 == 0:
#         print(lst)
        db.session.commit()
        
db.session.commit()
print('finish ')
# app_context.pop()        
        



In [119]:
# OrderSeq.__table__.columns.keys()
# db.session.rollback()

In [ ]:
# 导入dna_df 和 delivery_df 

# 导入dna_df表
dna_df['id'] = dna_df.index + 1 
# print(dna_df)


for ix,val in dna_df.iterrows():
    data = val.to_dict()
#     print(data)
    id =  data.get('id')
    if not id :
        continue   
    else:
#         print(val.get('id'))        
        
        o = OrderSeq.query.filter_by(id=id).first()
        if not o:
            print('can not find order: '+str(id))
            continue
         
#     if val.get('path') and not pd.isnull(val.get('path')):
        dnaInfo = DNAInfo()
        dnaInfo.from_dict(data)
             
        o.DNA_info.append(dnaInfo)
        db.session.add(dnaInfo)
    
#     old_id = data.get('id')
    
    i+= 1
    if i%100 == 0:
#         print(lst)
        db.session.commit()
        
db.session.commit()
print('finish ')
# app_context.pop()        

# db.session.rollback()

# 导入Delivery表
delivery_df['id'] = delivery_df.index + 1 

dna_df[id] = dna_df.index + 1 

for ix,val in delivery_df.iterrows():
    data = val.to_dict()
    id =  data.get('id')
    if not id :
        
        continue   
    else:
#         print(val.get('id'))        
        
        o = OrderSeq.query.filter_by(id=id).first()
        if not o:
            print('can not find order: '+str(id))
            continue
         
#     if val.get('path') and not pd.isnull(val.get('path')):
        delivery = Delivery()
        delivery.from_dict(data)
             
        o.delivery.append(delivery)
        db.session.add(delivery)
    
#     old_id = data.get('id')
    
    i+= 1
    if i%100 == 0:
#         print(lst)
        db.session.commit()
        
db.session.commit()
print('finish ')
# app_context.pop()   

In [124]:


# 订单关联到各个渠道
result_df.head()

,sample_code_x,detection_items,department,order_date,order_num,report_date,conclusion,receive_date,sample_code_format,random_num,event_flight,id,Unnamed: 0,date,code,gene_name,file_name,path,sample_code_y
0,ZSHC/2017/81,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,2018041664591,NaN,NaN,20180416,ZSHC-2017-81,64591,9C8555,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZSHC/2017/80,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,2018041664812,NaN,NaN,20180416,ZSHC-2017-80,64812,JD0472,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZSHC/2017/82,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,2018041645361,NaN,NaN,20180416,ZSHC-2017-82,45361,MF8026,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ZSHC/2017/83,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,2018041640074,NaN,NaN,20180416,ZSHC-2017-83,40074,JD5351,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ZSHC/2018/01,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,2018041679772,NaN,NaN,20180416,ZSHC-2018-01,79772,CA1704,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


由于订单里的渠道名不统一，且同一个渠道还有不同的名，在此统一一下：
```
珠港机场管理有限公司	珠港机场管理有限公司
重庆机场集团公司飞行区管理部	重庆机场集团公司飞行区管理部
郑州新郑国际机场场道管理部鸟击防范室	郑州新郑国际机场场道管理部鸟击防范室
浙江省义乌机场管理有限公司	浙江省义乌机场管理有限公司
长沙黄花国际机场分公司飞行区管理部	长沙黄花国际机场分公司飞行区管理部
徐州机场	徐州机场
武汉机场	武汉机场
天津滨海国际机场	天津滨海国际机场
深圳市机场股份有限公司	深圳机场
深圳机场股份有限公司	深圳机场
深圳机场	深圳机场
上海浦东国际机场	上海浦东国际机场
山东临沂机场	山东临沂机场
南通兴东国际机场	南通兴东国际机场
南宁吴圩国际机场	南宁吴圩国际机场
南宁机场修缮动力部	南宁吴圩国际机场
揭阳潮汕机场	揭阳潮汕机场
杭州萧山国际机场有限公司	杭州萧山国际机场
杭州萧山国际机场股份公司飞行区管理中心	杭州萧山国际机场
杭州萧山国际机场	杭州萧山国际机场
广州白云国际机场股份有限公司	广州白云国际机场
广州白云国际机场飞行区管理部生态管理部	广州白云国际机场
广州白云国际机场	广州白云国际机场
大连周水子国际机场	大连周水子国际机场
大连国际机场股份公司飞行区管理部	大连周水子国际机场
常州国际机场有限公司	常州国际机场有限公司
北京大兴国际机场	北京大兴国际机场

```

In [143]:
diff_lst = ['珠港机场管理有限公司','重庆机场集团公司飞行区管理部','郑州新郑国际机场场道管理部鸟击防范室','浙江省义乌机场管理有限公司','长沙黄花国际机场分公司飞行区管理部',
            '徐州机场','武汉机场','天津滨海国际机场','深圳市机场股份有限公司','深圳机场股份有限公司','深圳机场','上海浦东国际机场','山东临沂机场','南通兴东国际机场',
            '南宁吴圩国际机场','南宁机场修缮动力部','揭阳潮汕机场','杭州萧山国际机场有限公司','杭州萧山国际机场股份公司飞行区管理中心','杭州萧山国际机场','广州白云国际机场股份有限公司',
            '广州白云国际机场飞行区管理部生态管理部','广州白云国际机场','大连周水子国际机场','大连国际机场股份公司飞行区管理部','常州国际机场有限公司','北京大兴国际机场']

same_lst = ['珠港机场管理有限公司','重庆机场集团公司飞行区管理部','郑州新郑国际机场场道管理部鸟击防范室','浙江省义乌机场管理有限公司','长沙黄花国际机场分公司飞行区管理部',
'徐州机场','武汉机场','天津滨海国际机场','深圳机场','深圳机场','深圳机场','上海浦东国际机场','山东临沂机场','南通兴东国际机场','南宁吴圩国际机场','南宁吴圩国际机场','揭阳潮汕机场',
'杭州萧山国际机场','杭州萧山国际机场','杭州萧山国际机场','广州白云国际机场','广州白云国际机场','广州白云国际机场','大连周水子国际机场','大连周水子国际机场','常州国际机场有限公司','北京大兴国际机场']

airport_dict = dict(zip(diff_lst,same_lst))

print(airport_dict)

{'珠港机场管理有限公司': '珠港机场管理有限公司', '重庆机场集团公司飞行区管理部': '重庆机场集团公司飞行区管理部', '郑州新郑国际机场场道管理部鸟击防范室': '郑州新郑国际机场场道管理部鸟击防范室', '浙江省义乌机场管理有限公司': '浙江省义乌机场管理有限公司', '长沙黄花国际机场分公司飞行区管理部': '长沙黄花国际机场分公司飞行区管理部', '徐州机场': '徐州机场', '武汉机场': '武汉机场', '天津滨海国际机场': '天津滨海国际机场', '深圳市机场股份有限公司': '深圳机场', '深圳机场股份有限公司': '深圳机场', '深圳机场': '深圳机场', '上海浦东国际机场': '上海浦东国际机场', '山东临沂机场': '山东临沂机场', '南通兴东国际机场': '南通兴东国际机场', '南宁吴圩国际机场': '南宁吴圩国际机场', '南宁机场修缮动力部': '南宁吴圩国际机场', '揭阳潮汕机场': '揭阳潮汕机场', '杭州萧山国际机场有限公司': '杭州萧山国际机场', '杭州萧山国际机场股份公司飞行区管理中心': '杭州萧山国际机场', '杭州萧山国际机场': '杭州萧山国际机场', '广州白云国际机场股份有限公司': '广州白云国际机场', '广州白云国际机场飞行区管理部生态管理部': '广州白云国际机场', '广州白云国际机场': '广州白云国际机场', '大连周水子国际机场': '大连周水子国际机场', '大连国际机场股份公司飞行区管理部': '大连周水子国际机场', '常州国际机场有限公司': '常州国际机场有限公司', '北京大兴国际机场': '北京大兴国际机场'}


In [128]:
# 加入各个机场（渠道）信息
for i in range(len(same_lst)):
    c = Channel.query.filter_by(name=same_lst[i]).first()
    if not c :
        c = Channel('',same_lst[i])
        db.session.add(c)
db.session.commit()

In [130]:
# 创建用户信息：
for i in range(len(same_lst)):
    u = User.query.filter_by(username=same_lst[i]).first()
    if not u :
        u = User()
        u.username = same_lst[i]
        u.set_password('123456')
        
        c = Channel.query.filter_by(name=same_lst[i]).first()
        u.channel =c 
        db.session.add(u)
db.session.commit()

In [144]:

# 将订单与渠道管理

order_lst = OrderSeq.query.all()
i = 0
for o in order_lst:
#     print(o.department)
#     print(o)
#     i +=1
#     if i ==5:
#         break
        
    u = User.query.filter_by(username = airport_dict.get( o.department)).first()
    if u:
        o.owner = u
        
db.session.commit()
print('finish')

finish


In [145]:
o = OrderSeq.query.first()
print(o.owner)
# users = User.query.all()
# for u in users:
#     print(u.orders_seq.all())
print('--')

<User 杭州萧山国际机场>
--
